# [NAPARI](https://napari.org/)
## and [cellpose](https://cellpose-napari.readthedocs.io/en/latest/index.html) (installer [here](https://pypi.org/project/cellpose-napari/0.1.3/))

In [ ]:
## to install
# !pip install napari[all]
# !pip install QT
# !pip install napari[pyqt5]
# !pip install cellpose-napari==0.1.4
# !pip install napari-nikon-nd2
# !pip install magicgui
# !pip install napari-pyclesperanto-assistant
# !pip install napari-clusters-plotter #hdbscan failed
# !pip install napari-plot-profile
# !pip install napari-brightness-contrast
# !pip install napari-curtain
# !pip install napari-3d-ortho-viewer
# !pip install napari-manual-split-and-merge-labels
# !pip install napari-crop
# !pip install napari-stl-exporter

# !conda install -c conda-forge ipywidgets

## to upgrade:
# !pip install cellpose-napari==0.1.4
# !pip install cellpose-napari --upgrade
# !pip install cellpose --upgrade
# !pip install napari --upgrade
# !pip install mxnet-mkl

In [1]:
import napari
from magicgui import magicgui
from enum import Enum
import cellpose_napari
import cellpose
from cellpose import models
from napari.utils import nbscreenshot
from tifffile import imread
import numpy as np
from scipy import ndimage
from scipy.stats import mannwhitneyu
import napari_nikon_nd2
import os 
from magicgui import magicgui
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import importlib
import svgutils
from svgutils.compose import *
import ipywidgets
from tqdm import tqdm
import time
import sys
from plotParams import *


pBGMenu = ipywidgets.Dropdown(options=['Light', 'Dark'],value='Dark',disabled=False, layout=ipywidgets.Layout(width='20%', height='40px'))
pBGLabel = ipywidgets.widgets.Label('Select plotting style', layout=ipywidgets.Layout(width='20%', height='30px'))
pBG = ipywidgets.widgets.VBox([pBGLabel,pBGMenu])
pBG

In [2]:
applyPlotStyle(pBGMenu.value)
viewer = napari.Viewer()

Plotting style is Dark
Assistant skips harvesting pyclesperanto as it's not installed.


/Users/angueyraaristjm/opt/anaconda3/lib/python3.9/site-packages/napari_tools_menu/__init__.py:168: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


In [ ]:
# viewer.close()

In [ ]:
np.shape(viewer.layers[0].data)

In [ ]:
np.power(2,16)

***
# Index <a id='Index'></a>
***
- [Extraction of z planes for analysis](#zExtract)
- [Segmentation with cellpose](#cellSeg)
- [Manual correction of segmentation](#manualCuration)
- [Create thumbnails _WIP_](#thumbnails)
- [Quantification](#quantification)

***
## Extract layers from z-stacks<a id='zExtract' name='zExtract'>∮</a>
***
[Back to Index](#Index)

In [ ]:
# getting list of files
dPath = "/Volumes/angueyraNEI/LiImaging/zf/20211215_lrrfip1aF0_m2G_s2C/"
gene = 'lrrfip1a'
#### !ls $dPath/*$gene*[002,004].nd2
fList = !ls $dPath/*$gene*[002,004].nd2
fList = list(map(lambda st: str.replace(st, dPath + '/', 'fPath = \"'), fList))
fList = list(map(lambda st: str.replace(st, '.nd2', '\"; gene = \'' + gene + '\'; '), fList))
# print(*fList, sep = "\n")

In [ ]:
# clear viewer
viewer.layers.select_all(); viewer.layers.remove_selected()

In [11]:
# clear viewer
viewer.layers.select_all(); viewer.layers.remove_selected()

# analysis directory
dAnalysis = "/Users/angueyraaristjm/Documents/eelImaging/tempAnalysis/test/"

# open file
dPath = "/Volumes/angueyra04/LiImaging/20230116_nr2e3nr2f1bF0s_m2G_s2C/"

fPath = "20230116_m2G_s2C_F0nr2e3nr2f1b_L01_004"; gene = 'wt'; zlims = [7,9];


photoreceptors = ['M','S']

# create subdirectory for analysis
dOut = dAnalysis + gene + '/' + fPath + '/'
if (os.path.isdir(dOut)==False):
    os.mkdir(dOut)
    print('Created output directory')

print('Canvas clear')

Canvas clear


In [12]:
# load whole stack to figure out best planes for cellpose
print('Loading files...')
viewer.open((dPath+fPath + ".nd2"))
viewer.layers.remove(viewer.layers[len(viewer.layers)-1]); viewer.layers.select_next(); # remove transmitted detector image

if len(viewer.layers)==2: # GFP and RFP channel (no DAPI)
    nChannels = 2
    viewer.layers[0].colormap = 'green'; viewer.layers[0].name = photoreceptors[0]; viewer.layers[0].opacity = 1.00
    viewer.layers[1].colormap = 'magenta'; viewer.layers[1].name = photoreceptors[1]; viewer.layers[1].opacity = 1.00
elif len(viewer.layers)==3: #DAPI
    nChannels = 3
    viewer.layers[0].colormap = 'bop blue'; viewer.layers[0].name = 'N'; viewer.layers[0].opacity = 1.00
    viewer.layers[1].colormap = 'green'; viewer.layers[1].name = photoreceptors[0];
    viewer.layers[2].colormap = 'magenta'; viewer.layers[2].name = photoreceptors[1]

print('Loaded: ' + fPath)

Loading files...
Loaded: 20230116_m2G_s2C_F0nr2e3nr2f1b_L01_004


In [ ]:
# make mips (and remove any previous ones)
zlims = [7,9];
# zlims = [2,13];
if len(viewer.layers)>nChannels:
    for l in viewer.layers[nChannels:]:
        viewer.layers.remove(l)

for l in [viewer.layers[0],viewer.layers[1],viewer.layers[2]]: # DAPI and FP channels
    l.visible = False
    viewer.add_image(l.data[zlims[0]:zlims[1]].max(axis=0), blending='additive', colormap = l.colormap, name = l.name + "_mip")

In [ ]:
# when things look good, save in folder for batch cellpose analysis
for pr in photoreceptors:
    l = viewer.layers[pr + '_mip']; l.save(dOut + l.name + '.tiff')
l = viewer.layers['N_mip']; l.save(dOut + l.name + '.tiff')
print('MIP layers saved for ' + fPath)

### Establish landmarks in whole-eye MIP

In [ ]:
nMIP = 3 * 2
for pr in photoreceptors:
    viewer.layers[pr + '_mip'].scale = [1.5/8,1.5/8]; viewer.layers[pr + '_mip'].translate = [0,1024-(1024*1.5/8)]
viewer.layers['N_mip'].visible = False;

# Load whole-eye MIP to establish landmarks
print('Loading eye mip...')
if str.endswith(fPath,'002'):
    mipPath = fPath[:-1] + "1_mip";
elif str.endswith(fPath,'004'):
    mipPath = fPath[:-1] + "3_mip";
viewer.open((dPath+mipPath + ".nd2"))

viewer.layers.remove(viewer.layers[len(viewer.layers)-1]) # remove transmitted detector image
viewer.layers.select_next();

if len(viewer.layers)==2+nMIP: # no DAPI, GFP and RFP
    nChannels = 2
    viewer.layers[nMIP].colormap = 'green'
    viewer.layers[nMIP].name = 'eyeGreen'
    viewer.layers[nMIP+1].colormap = 'magenta'
    viewer.layers[nMIP+1].name = 'eyeRed'
if len(viewer.layers)==2+nMIP: # DAPI and RFP channel (single positive larvae with bad immunos)
    nChannels = 2
    viewer.layers[nMIP+0].colormap = 'green'
    viewer.layers[nMIP+0].name = 'eyeGreen'
    viewer.layers[nMIP+0].opacity = 0.66
    viewer.layers[nMIP+1].colormap = 'magenta'
    viewer.layers[nMIP+1].name = 'eyeRed'
elif len(viewer.layers)==3+nMIP: #DAPI
    nChannels = 3
    viewer.layers[nMIP+0].colormap = 'bop blue'
    viewer.layers[nMIP+0].name = 'eyeN'
    viewer.layers[nMIP+0].opacity = 0.33
    viewer.layers[nMIP+1].colormap = 'green'
    viewer.layers[nMIP+1].name = 'eyeGreen'
    viewer.layers[nMIP+2].colormap = 'magenta'
    viewer.layers[nMIP+2].name = 'eyeRed'


print('Loaded: ' + fPath)
eyeMarks = viewer.add_points(size=20, name = 'eyeMarks', symbol="cross", face_color='#ffffff80')
viewer.layers.select_next();

# Save eye landmarks
eyeMarksButton = ipywidgets.Button(description='Save eyeMarks', layout=ipywidgets.Layout(width='200px', height='100px'))
out = ipywidgets.Output()
def eyeMClick(_):
    with out:
        l = viewer.layers['eyeMarks']; l.save(dOut + l.name + '.csv');
        print('eyeMarks layers saved for ' + fPath)
    # clear viewer
    viewer.layers.select_all()
    viewer.layers.remove_selected()
    eyeMarksButton.disabled = True # to prevent double clicking warnings. All sales are final!
    
eyeMarksButton.on_click(eyeMClick)

print("Make 6 landmark points then click button to save")

ipywidgets.VBox([eyeMarksButton,out])

- Use eyeMarks layer to mark:
        - Landmark #1: Center of FOV (where zoom in is centered)
        - Landmark #2: Center of Optic Nerve Head 
        - Landmark #3: Dorsal edge of eye
        - Landmark #4: Ventral edge of eye
        - Landmark #5: Temporal edge of eye (strike zone side)
        - Landmark #6: Nasal edge of eye
- Decide if stack should be excluded (and stop analysis here and make note in csv summary file)

### [Go back and run next one &uarr;](#zExtract)
---

***
## Segmentation using cellpose<a id='cellSeg'>∮</a>
***
[Back to Index](#Index)

## Run cellpose as batch analysis
Usually good parameters are :
1. xOPS:GFP: diameter = 20; flow_threshold = 0.4 and mask_threshold = 0.0
1. sws1:mCherry: diameter = 40; flow_threshold = 0.4 and mask_threshold = 0.0
1. sws2:mCherry:
1. mws2:GFP:
1. thrb:tdTomato:
    
flow_threshold = 0.6 will give more ROIs

In [ ]:
# clear viewer
viewer.layers.select_all(); viewer.layers.remove_selected()

In [13]:

# # analysis directory
# dAnalysis = "/Users/angueyraaristjm/Documents/eelImaging/tempAnalysis/test/"

# # open file
# dPath = "/Volumes/angueyra04/LiImaging/20230116_nr2e3nr2f1bF0s_m2G_s2C/"


### collect file paths
wt_gene = 'wt';
wt_fPaths = [
    '20230116_m2G_s2C_F0nr2e3nr2f1b_L01_004',

]

# define cellpose params
cpParams = {
    'model' : 'cyto2', # default is 'cyto' or 'cyto2'
    'net_avg' : True,
    'channels' : [0,0], #single channel without nucleus info
    'diameterR' : 25,
    'diameterC' : 40,
    'flow_threshold' : 0.5,
    'cellprob_threshold' : 0.0
}

# define model to use (e.g. 'cyto2')
model = models.Cellpose(gpu=False, model_type=cpParams['model'])

In [15]:
print('Starting analysis:\n')
with tqdm(total=len(wt_fPaths)*3, file=sys.stdout) as progBar:
    for fPath in wt_fPaths:
        print('\n' + fPath)
        print('\t S cones')
        startTime = time.time()
        dOut = dAnalysis + wt_gene + '/' + fPath + '/' + 'S_mip.tiff'
        img = cellpose.io.imread(dOut)
        masks, flows, styles, diams = model.eval(img, 
                                                 diameter=cpParams['diameterR'], channels=cpParams['channels'],
                                                 do_3D=False, net_avg = cpParams['net_avg'], interp = True,
                                                 flow_threshold = cpParams['flow_threshold'], cellprob_threshold = cpParams['cellprob_threshold'])
        cellpose.io.masks_flows_to_seg(img, masks, flows, diams, dOut, cpParams['channels']) # save results
        endTime = time.time()
        print ('\t Time elapsed: {elapsedTime} s\n'.format(elapsedTime = int(endTime - startTime)))
        progBar.update(1)
        
        dOut = dAnalysis + wt_gene + '/' + fPath + '/' + 'M_mip.tiff'
        img = cellpose.io.imread(dOut)
        masks, flows, styles, diams = model.eval(img, 
                                                 diameter=cpParams['diameterC'], channels=cpParams['channels'],
                                                 do_3D=False, net_avg = cpParams['net_avg'], interp = True,
                                                 flow_threshold = cpParams['flow_threshold'], cellprob_threshold = cpParams['cellprob_threshold'])
        cellpose.io.masks_flows_to_seg(img, masks, flows, diams, dOut, cpParams['channels']) # save results
        endTime = time.time()
        print ('\t Time elapsed: {elapsedTime} s\n'.format(elapsedTime = int(endTime - startTime)))
        progBar.update(1)
        
        dOut = dAnalysis + wt_gene + '/' + fPath + '/' + 'N_mip.tiff'
        img = cellpose.io.imread(dOut)
        masks, flows, styles, diams = model.eval(img, 
                                                 diameter=cpParams['diameterC'], channels=cpParams['channels'],
                                                 do_3D=False, net_avg = cpParams['net_avg'], interp = True,
                                                 flow_threshold = cpParams['flow_threshold'], cellprob_threshold = cpParams['cellprob_threshold'])
        cellpose.io.masks_flows_to_seg(img, masks, flows, diams, dOut, cpParams['channels']) # save results
        endTime = time.time()
        print ('\t Time elapsed: {elapsedTime} s\n'.format(elapsedTime = int(endTime - startTime)))
        progBar.update(1)
    
print('\nFinished cellpose batch analysis')

# print('Starting analysis:\n')
# with tqdm(total=len(cr_fPaths)*3, file=sys.stdout) as progBar:
#     for fPath in cr_fPaths:
#         print('\n' + fPath)
#         print('\t S cones')
#         startTime = time.time()
#         dOut = dAnalysis + cr_gene + '/' + fPath + '/' + 'S_mip.tiff'
#         img = cellpose.io.imread(dOut)
#         masks, flows, styles, diams = model.eval(img, 
#                                                  diameter=cpParams['diameterR'], channels=cpParams['channels'],
#                                                  do_3D=False, net_avg = cpParams['net_avg'], interp = True,
#                                                  flow_threshold = cpParams['flow_threshold'], cellprob_threshold = cpParams['cellprob_threshold'])
#         cellpose.io.masks_flows_to_seg(img, masks, flows, diams, dOut, cpParams['channels']) # save results
#         endTime = time.time()
#         print ('\t Time elapsed: {elapsedTime} s\n'.format(elapsedTime = int(endTime - startTime)))
#         progBar.update(1)
        
#         dOut = dAnalysis + cr_gene + '/' + fPath + '/' + 'M_mip.tiff'
#         img = cellpose.io.imread(dOut)
#         masks, flows, styles, diams = model.eval(img, 
#                                                  diameter=cpParams['diameterC'], channels=cpParams['channels'],
#                                                  do_3D=False, net_avg = cpParams['net_avg'], interp = True,
#                                                  flow_threshold = cpParams['flow_threshold'], cellprob_threshold = cpParams['cellprob_threshold'])
#         cellpose.io.masks_flows_to_seg(img, masks, flows, diams, dOut, cpParams['channels']) # save results
#         endTime = time.time()
#         print ('\t Time elapsed: {elapsedTime} s\n'.format(elapsedTime = int(endTime - startTime)))
#         progBar.update(1)
        
#         dOut = dAnalysis + cr_gene + '/' + fPath + '/' + 'N_mip.tiff'
#         img = cellpose.io.imread(dOut)
#         masks, flows, styles, diams = model.eval(img, 
#                                                  diameter=cpParams['diameterC'], channels=cpParams['channels'],
#                                                  do_3D=False, net_avg = cpParams['net_avg'], interp = True,
#                                                  flow_threshold = cpParams['flow_threshold'], cellprob_threshold = cpParams['cellprob_threshold'])
#         cellpose.io.masks_flows_to_seg(img, masks, flows, diams, dOut, cpParams['channels']) # save results
#         endTime = time.time()
#         print ('\t Time elapsed: {elapsedTime} s\n'.format(elapsedTime = int(endTime - startTime)))
#         progBar.update(1)
    
# print('\nFinished cellpose batch analysis')

Starting analysis:

  0%|                                                                       | 0/3 [00:00<?, ?it/s]
20230116_m2G_s2C_F0nr2e3nr2f1b_L01_004
	 S cones
	 Time elapsed: 23 s

 33%|█████████████████████                                          | 1/3 [00:23<00:47, 23.73s/it]	 Time elapsed: 39 s

 67%|██████████████████████████████████████████                     | 2/3 [00:39<00:19, 19.31s/it]	 Time elapsed: 60 s

100%|███████████████████████████████████████████████████████████████| 3/3 [01:00<00:00, 20.32s/it]

Finished cellpose batch analysis


***
## Manual correction of cellpose segmentation<a id='manualCuration'>∮</a>
***
[Back to Index](#Index)

In [ ]:
# viewer = napari.Viewer()
viewer.layers.select_all(); viewer.layers.remove_selected()

In [16]:
# clear viewer
viewer.layers.select_all(); viewer.layers.remove_selected()


# # analysis directory
# dAnalysis = "/Users/angueyraaristjm/Documents/eelImaging/tempAnalysis/test/"

# # open file
# dPath = "/Volumes/angueyra04/LiImaging/20230116_nr2e3nr2f1bF0s_m2G_s2C/"



fPath = "20230116_m2G_s2C_F0nr2e3nr2f1b_L01_004"; gene = 'wt'; # ok


# define subdirectory for analysis
dOut = dAnalysis + gene + '/' + fPath + '/'
print('Viewer cleared...')

# clear key binds
@viewer.bind_key('k', overwrite=True)
def toggle_sel(viewer):
    ...
@viewer.bind_key('b', overwrite=True)
def toggle_sel(viewer):
    ...
@viewer.bind_key('Shift-x', overwrite=True)
def removeLabel(viewer):
    ...

photoreceptors = ['M','S','N']
colorList = ['green','magenta','bop blue','orange']

# load mips
mipO = 0.65
for pr,cl in zip(photoreceptors,colorList):
    viewer.open(dOut + pr + "_mip.tiff", plugin='builtins', colormap = cl, blending='additive', opacity=mipO);

# load segmentations
for pr in photoreceptors:
    segData = np.load(dOut + pr + '_mip' + '_seg.npy', allow_pickle=True).item()
    viewer.add_labels(segData['masks'], name = pr + '_seg',blending='additive');
    viewer.layers[pr + '_seg'].preserve_labels = True;

# tweak display to help with segmentation
pr = photoreceptors[0]
# viewer.layers[pr + '_mip'].opacity=0.40;viewer.layers[pr + '_mip'].gamma=1.3;viewer.layers[pr + '_mip'].contrast_limits=[150,2000]

pr = photoreceptors[1]
viewer.layers[pr + '_mip'].visible = True; viewer.layers[pr + '_seg'].visible = False;
pr = photoreceptors[2]
viewer.layers[pr + '_mip'].visible = False; viewer.layers[pr + '_seg'].visible = False;

print('Loaded  ' + fPath + ' !\n Now fix it and save it!')

#define useful keyboard shortcuts

pr = photoreceptors[0]
@viewer.bind_key('Shift-x', overwrite=True)
def removeLabel(viewer):
    lname = pr + '_seg'
    tempd = viewer.layers[lname].data
    tempd[tempd == viewer.layers[lname].selected_label]=0
    viewer.layers[lname].data = tempd
    print('Cut!')
print('removeLabel on {0}_seg ("Shift-X")'.format(photoreceptors[0]))

@viewer.bind_key('k', overwrite=True)
def toggle_sel(viewer):
    lname = pr + '_seg'
    if (viewer.layers[lname].preserve_labels == True):
        viewer.layers[lname].preserve_labels = False
    elif (viewer.layers[lname].preserve_labels == False):
        viewer.layers[lname].preserve_labels = True
print('toggle {0}_seg preserve_labels ("k")'.format(pr))
        
@viewer.bind_key('b', overwrite=True)
def toggle_sel(viewer):
    lname = photoreceptors[1] + '_mip'
    if (viewer.layers[lname].visible == True):
        viewer.layers[lname].visible = False
    elif (viewer.layers[lname].visible == False):
        viewer.layers[lname].visible = True
print('toggle {0}_mip visibility ("B")'.format(photoreceptors[1]))

@viewer.bind_key('n', overwrite=True)
def toggle_sel(viewer):
    lname = pr + '_mip'
    if (viewer.layers[lname].visible == True):
        viewer.layers[lname].visible = False
    elif (viewer.layers[lname].visible == False):
        viewer.layers[lname].visible = True
print('toggle {0}_mip visibility ("N")'.format(pr))

@viewer.bind_key('+', overwrite=True)
def new_label(viewer):
    """Set the currently selected label to the largest used label plus one."""
    lname = pr + '_seg'
    viewer.layers[lname].selected_label = viewer.layers[lname].data.max() + 1
print('Add new label in {0}_seg visibility ("+"|"M")'.format(pr))

Viewer cleared...
Loaded  20230116_m2G_s2C_F0nr2e3nr2f1b_L01_004 !
 Now fix it and save it!
removeLabel on M_seg ("Shift-X")
toggle M_seg preserve_labels ("k")
toggle S_mip visibility ("B")
toggle M_mip visibility ("N")
Add new label in M_seg visibility ("+"|"M")


Traceback (most recent call last):
  File "/Users/angueyraaristjm/opt/anaconda3/lib/python3.9/site-packages/napari/_qt/layer_controls/qt_labels_controls.py", line 357, in changeSelection
    self.layer.selected_label = value
  File "/Users/angueyraaristjm/opt/anaconda3/lib/python3.9/site-packages/napari/layers/labels/labels.py", line 610, in selected_label
    self._selected_color = self.get_color(selected_label)
  File "/Users/angueyraaristjm/opt/anaconda3/lib/python3.9/site-packages/napari/layers/labels/labels.py", line 978, in get_color
    val = self._raw_to_displayed(np.array([label]))
  File "/Users/angueyraaristjm/opt/anaconda3/lib/python3.9/site-packages/napari/layers/labels/labels.py", line 946, in _raw_to_displayed
    image = self._color_lookup_func(raw_modified, self._selected_label)
  File "/Users/angueyraaristjm/opt/anaconda3/lib/python3.9/site-packages/napari/layers/labels/labels.py", line 818, in _lookup_with_index
    colors[selected_label] = low_discrepancy_image(
Ind

---

### resave curated segmentation after napari-ing around (M_mip first)

In [ ]:
# Doing 2 channel segmentation
baseName = photoreceptors[0]
lname = baseName + '_seg'
l = viewer.layers[baseName + '_seg']; l.save(dOut + l.name + '_curated.tiff')
print('Done with ' + lname + ' for ' + fPath)

viewer.layers[photoreceptors[1] + '_mip'].visible = True
viewer.layers[photoreceptors[1] + '_seg'].visible = True
viewer.layers[photoreceptors[1] + '_seg'].contour = 0
viewer.layers[photoreceptors[0] + '_mip'].visible = False
viewer.layers[photoreceptors[0] + '_seg'].visible = False
# viewer.layers['N_mip'].visible = True

viewer.add_points(size=10, name = 'U_missing', symbol="o", face_color='#B73AB9', edge_color='#FFFFFF')
viewer.add_points(size=10, name = 'L_missing', symbol="o", face_color='#CE2A22', edge_color='#FFFFFF')


@viewer.bind_key('Shift-x', overwrite=True)
def removeLabel(viewer):
    lname = photoreceptors[1] + '_seg'
    tempd = viewer.layers[lname].data
    tempd[tempd == viewer.layers[lname].selected_label]=0
    viewer.layers[lname].data = tempd
    print('Cut!')
print('removeLabel on {0}_seg ("Shift-X")'.format(photoreceptors[1]))

@viewer.bind_key('k', overwrite=True)
def toggle_sel(viewer):
    lname = photoreceptors[1] + '_seg'
    if (viewer.layers[lname].preserve_labels == True):
        viewer.layers[lname].preserve_labels = False
    elif (viewer.layers[lname].preserve_labels == False):
        viewer.layers[lname].preserve_labels = True
print('toggle {0}_seg preserve_labels ("k")'.format(photoreceptors[1]))

@viewer.bind_key('b', overwrite=True)
def toggle_sel(viewer):
    lname = photoreceptors[0] + '_mip'
    if (viewer.layers[lname].visible == True):
        viewer.layers[lname].visible = False
    elif (viewer.layers[lname].visible == False):
        viewer.layers[lname].visible = True
print('toggle {0}_mip visibility ("B")'.format(photoreceptors[0]))

        
@viewer.bind_key('n', overwrite=True)
def toggle_sel(viewer):
    lname = photoreceptors[1] + '_mip'
    if (viewer.layers[lname].visible == True):
        viewer.layers[lname].visible = False
    elif (viewer.layers[lname].visible == False):
        viewer.layers[lname].visible = True
print('toggle {0}_mip visibility ("N")'.format(photoreceptors[1]))

@viewer.bind_key('+', overwrite=True)
def new_label(viewer):
    """Set the currently selected label to the largest used label plus one."""
    lname = photoreceptors[1] + '_seg'
    viewer.layers[lname].selected_label = viewer.layers[lname].data.max() + 1
print('Add new label in {0}_seg visibility ("+"|"M")'.format(photoreceptors[1]))

### Save S-cone segmentation (and UV-cone nuclei if possible)

In [ ]:
baseName = photoreceptors[1]
lname = baseName + '_seg'
l = viewer.layers[baseName + '_seg']; l.save(dOut + l.name + '_curated.tiff')
print('Done with ' + lname + ' for ' + fPath)

# If NUCS mip revealed UV cones
baseName = 'U'
lname = baseName + '_missing'
l = viewer.layers[lname]; l.save(dOut + lname);
print("Image has {0} {1}-cone nuclei".format(len(viewer.layers[lname].data),baseName))

# If NUCS mip also revealed L cones
baseName = 'L'
lname = baseName + '_missing'
l = viewer.layers[lname]; l.save(dOut + lname);
print("Image has {0} {1}-cone nuclei".format(len(viewer.layers[lname].data),baseName))

### Reload curated segmentations after saving

In [ ]:
# remove all except mips
viewer.layers.remove(viewer.layers['U_missing'])
viewer.layers.remove(viewer.layers['L_missing'])

n_mips = 3;
if len(viewer.layers)>n_mips:
    for l in viewer.layers[n_mips:]:
        viewer.layers.remove(l)

viewer.open(dOut + photoreceptors[0] + '_seg_curated.tiff', name=photoreceptors[0] + '_seg', plugin='builtins',blending='additive');
viewer.layers[photoreceptors[0] + '_seg'].contour = 5
viewer.layers[photoreceptors[0] + '_mip'].visible = True
viewer.open(dOut + photoreceptors[1] + '_seg_curated.tiff', name=photoreceptors[1] + '_seg', plugin='builtins',blending='additive');

print(fPath)
nGreen = len(np.unique(viewer.layers[photoreceptors[0] + '_seg'].data))-1
print("Image has {0} {1} cones".format(nGreen,photoreceptors[0]))
nRed = len(np.unique(viewer.layers[photoreceptors[1] + '_seg'].data))-1
print("Image has {0} {1} cones".format(nRed,photoreceptors[1]))

### [Go back and run next one &uarr;](#manualCuration)
---

### reload mip and eye marks in case there are doubts

In [ ]:

nMIP = 3 + 1
viewer.layers['R_seg'].visible = False;
viewer.layers['R_mip'].visible = False;
viewer.layers['N_mip'].visible = False;
viewer.layers['C_mip'].visible = False;

# Load whole-eye MIP to establish landmarks
print('Loading eye mip...')
if str.endswith(fPath,'002'):
    mipPath = fPath[:-1] + "1_mip";
elif str.endswith(fPath,'004'):
    mipPath = fPath[:-1] + "3_mip";
viewer.open((dPath+mipPath + ".nd2"))

viewer.layers.remove(viewer.layers[len(viewer.layers)-1]) # remove transmitted detector image
viewer.layers.select_next();

if len(viewer.layers)==2+nMIP: # no DAPI, GFP and RFP
    nChannels = 2
    viewer.layers[nMIP].colormap = 'green'
    viewer.layers[nMIP].name = 'eyeR'
    viewer.layers[nMIP+1].colormap = 'magenta'
    viewer.layers[nMIP+1].name = 'eyeC'
if len(viewer.layers)==2+nMIP: # DAPI and RFP channel (single positive larvae with bad immunos)
    nChannels = 2
    viewer.layers[nMIP+0].colormap = 'green'
    viewer.layers[nMIP+0].name = 'eyeR'
    viewer.layers[nMIP+0].opacity = 0.66
    viewer.layers[nMIP+1].colormap = 'magenta'
    viewer.layers[nMIP+1].name = 'eyeC'
elif len(viewer.layers)==3+nMIP: #DAPI
    nChannels = 3
    viewer.layers[nMIP+0].colormap = 'gray'
    viewer.layers[nMIP+0].name = 'eyeN'
    viewer.layers[nMIP+0].opacity = 0.33
    viewer.layers[nMIP+1].colormap = 'green'
    viewer.layers[nMIP+1].name = 'eyeR'
    viewer.layers[nMIP+2].colormap = 'magenta'
    viewer.layers[nMIP+2].name = 'eyeC'

eyeMarksData  = np.loadtxt(dOut + 'eyeMarks.csv',delimiter=',',skiprows=1,usecols=[1,2])
eyeMarks = viewer.add_points(eyeMarksData,size=20, name = 'eyeMarks', symbol="cross", face_color='#ffffff80')

In [ ]:
# # manual merging of labels
# mergeLabels = [167,168]
# # lname = 'M_seg'
# lname = 'L_seg'
# if viewer.layers[lname].visible == False:
#     print('Layer not active')
# else:
#     tempd = viewer.layers[lname].data
#     tempd[tempd == mergeLabels[1]]=mergeLabels[0]
#     viewer.layers[lname].data = tempd
#     print('Merged in ' + lname)

In [ ]:
# # manually delete label
# selLabel = 49
# lname = 'L_seg'
# tempd = viewer.layers[lname].data
# tempd[tempd == selLabel]=0
# viewer.layers[lname].data = tempd

# for selLabel in np.arange(15,300):
#     lname = 'L_seg'
#     tempd = viewer.layers[lname].data
#     tempd[tempd == selLabel]=0
#     viewer.layers[lname].data = tempd

***
## Quantification<a id='quantification'></a>
***
[Back to Index](#Index)

### Read csv created during analysis and create bar plot

In [ ]:
pBGMenu

In [ ]:
#reapply plotting style
applyPlotStyle(pBGMenu.value)

In [ ]:
# Created csv manually during analysis
dPath = "/Users/angueyraaristjm/Documents/eelImaging/tempAnalysis/CRnr2e3F0s/xOGaCT/"
fName = "CRnr2e3F0s_xOGaCT_Counts.csv"

dPath = "/Users/angueyraaristjm/Library/CloudStorage/OneDrive-NationalInstitutesofHealth/zf/Analysis/CRnr2e3F0s/"
# dPath = "/Users/angueyraaristjm/Documents/eelImaging/tempAnalysis/CRnr2e3F0s/m2Gs2C/"
dPath = '/Users/angueyraaristjm/Library/CloudStorage/OneDrive-NationalInstitutesofHealth/zf/Analysis/CRlrrfip1aF0s/'
fName = "CRlrrfip1aF0s_combinedCounts.csv"

df = pd.read_csv(dPath + fName)
df

In [ ]:
geneList = ['wt','lrrfip1a']
nGenes = np.size(geneList)
photoreceptors = ['R','U','S','M','L']
# photoreceptors = ['S','M']

plotname = ''
fH, axH = plt.subplots(figsize= [12,8])

barW = nGenes+2; # bar width
barD = nGenes+1; # bar distance whitin 1 photoreceptor subtype
barP = -nGenes+1; # position in x-axis
barStep = 0.9; # distance between photoreceptor groups
barPos = []; #array to save bar positions


j=barP;
for gene in geneList:
    i=0;
    j=j+1;
    for pr in photoreceptors:
        i = i+barStep;
        countData = df[(df['genotype']==gene) & (df['excludeFlag']==False)][pr]
        jitter = np.random.randn(len(countData))*0.03
#         textindent = 1/(2.5*barD);
        textindent = 0.10;
        pH = plt.bar([i+j/barD], np.mean(countData), width=1/barW, color=lighten_color(zfC[pr],1), linewidth = 2, edgecolor = lighten_color(zfC[pr],1)); #color=zfC[pr], 
#         pH = plt.bar([i+j/barD], np.mean(countData), yerr = sem(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr][~np.isnan(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr])], ddof = 0), align='center', ecolor='black', capsize=10, width=1/barW, color=zfC[pr], edgecolor = zfC[pr]);
#         pH = plt.bar([i+j/barD], np.mean(countData), yerr = np.std(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr][~np.isnan(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr])], ddof = 0), align='center', ecolor='black', capsize=10, width=1/barW, color=zfC[pr], edgecolor = zfC[pr]);
       # pH = plt.bar([i+j/barD], np.mean(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), yerr = np.std(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), ecolor=lighten_color(zfC[pr],1.2), capsize=10, width=1/barW, color=zfC[pr], edgecolor = zfC[pr]);
        #pH = plt.errorbar([i+j/barD], np.mean(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), yerr = np.std(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), ecolor=lighten_color(zfC[pr],1.1), elinewidth=2, capsize=5, capthick=2, barsabove= True)
#         pH = plt.scatter(np.ones(len(countData))*[i+j/barD]+jitter, countData, color=lighten_color(zfC[pr],2/3), zorder=2, marker = zfGm[gene], edgecolor='#ffffff', linewidth=0.5, alpha = 0.4);
        pH = plt.errorbar(i+j/barD, np.mean(countData), yerr = [[0],[np.std(countData)]], ecolor=lighten_color(zfC[pr],1), elinewidth=3, capsize=8, capthick=3, zorder=9)
        # plt.text((i+j/barD)-textindent, 3, prLabel[pr], font_properties=font_prop, fontsize=20, ha='left', alpha=0.6)
        pH = plt.scatter(np.ones(len(countData))*[i+j/barD]+jitter, countData, color='#ffffff', zorder=8, marker = 'o', s=50, edgecolor='#ffffff', linewidth=0.5, alpha = .6);
        barPos = np.append(barPos,(i+j/barD))

formatFigure(fH, axH, pH)
axH.set_xticks(np.sort(barPos));

axH.set_xticklabels(geneList * len(photoreceptors));
# axH.set_ylim([0,500]); # this was 400

# savePath = 'C:/Users/pataklk/OneDrive - National Institutes of Health/zf/F0_Analysis/CRfoxq2F0/'
# savePath = "/Users/angueyraaristjm/OneDrive - National Institutes of Health/zf/F0_Analysis/CRfoxq2F0/"
# savePath = "/Users/angueyraaristjm/Documents/LiLab/Presentations/revealjs/resources/20211008_UCLA/"
# plt.savefig(savePath + "Counts_foxq2.png", transparent=True, format="png", bbox_inches = "tight")
# plt.savefig(savePath + "Counts_foxq2_B.svg", transparent=True, format="svg", bbox_inches = "tight")

### Stats

### Read csv created during analysis and create bar plot

In [ ]:
geneList = ['wt','lrrfip1a']
photoreceptors = ['R','U','S','M']

print('{0} vs. {1}:'.format(geneList[0],geneList[1]))

for pr in photoreceptors:
    # get counts for each photoreceptor subtype and exclude NaNs
    wtCount = df[(df['genotype']==(geneList[0]))&(df['excludeFlag']==False)][pr]
    wtCount = wtCount[~np.isnan(wtCount)]
    
    crCount = df[(df['genotype']==(geneList[1]))&(df['excludeFlag']==False)][pr]
    crCount = crCount[~np.isnan(crCount)]
    
    u, p = mannwhitneyu(wtCount, crCount)
    print('\t{0}:'.format(pr))
    print('\t\tU = {0:.3f}, p = {1:.5f}, nEyes: wt = {2:.0f}; cr = {3:.0f}'.format(u,p,len(wtCount),len(crCount)))


In [ ]:
#gene Colors
zfC = {
    'R'  : '#7d7d7d',
    'U' : '#B73AB9',
    'S' : '#4364F6',
    'M' : '#59CB3B',
    'L' : '#CE2A22',
}

zfG = {
    'wt' : '#000000',
    'tbx2a' : '#ab266b',
    'tbx2b' : '#421f8e',
    'foxq2' : '#001dd6',
    'nr2e3' : '#7d7d7d',
}

zfGm = {
    'wt' : 'o',
    'tbx2a' : 'P',
    'tbx2b' : 'X',
    'nr2e3' : '+',
    
}

prLabel = {
    'R'  : 'Rods',
    'U' : 'UV',
    'S' : 'S',
    'M' : 'M',
    'L' : 'L',
}


def formatFigureMain(figH, axH, plotH):
#     font_path = 'C:/Users/pataklk/Documents/Frag_Analysis_Code/Avenir.ttc'
    font_path = '/System/Library/Fonts/Avenir.ttc'
    fontTicks = font_manager.FontProperties(fname=font_path, size=30) # was 18
    fontLabels = font_manager.FontProperties(fname=font_path, size=36) # was 22
    fontTitle = font_manager.FontProperties(fname=font_path, size=30) # was 28 
    axH.set_xscale('linear')
    axH.spines['top'].set_visible(False)
    axH.spines['right'].set_visible(False)
    
    for label in (axH.get_xticklabels() + axH.get_yticklabels()):
        label.set_fontproperties(fontTicks)
    axH.set_xlabel(axH.get_xlabel(), fontproperties = fontTicks)
    axH.set_ylabel(axH.get_ylabel(), fontproperties = fontTicks)
    return fontLabels

def formatFigure(figH, axH, plotH):
    fontLabels = formatFigureMain(figH, axH, plotH)
#     axH.set_xlabel('wt vs. cr', fontproperties=fontLabels)
    axH.set_ylabel('cells per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.xaxis.set_tick_params(rotation=45)

def formatFigureRvU(figH, axH, plotH):
    fontLabels = formatFigureMain(figH, axH, plotH)
    axH.set_xlabel('Rods per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.set_ylabel('UV cones per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.xaxis.set_tick_params(rotation=45)
    
def formatFigureMvS(figH, axH, plotH):
    fontLabels = formatFigureMain(figH, axH, plotH)
    axH.set_xlabel('M cones per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.set_ylabel('S cones per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.xaxis.set_tick_params(rotation=45)

    
def lighten_color(color, amount=0.5):
    """
    Lightens the given color by multiplying (1-luminosity) by the given amount.
    Input can be matplotlib color string, hex string, or RGB tuple.

    Examples:
    >> lighten_color('g', 0.3)
    >> lighten_color('#F034A3', 0.6)
    >> lighten_color((.3,.55,.1), 0.5)
    """
    import colorsys
    try:
        c = matplotlib.colors.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*matplotlib.colors.to_rgb(c))
    return matplotlib.colors.rgb2hex(colorsys.hls_to_rgb(c[0], 1 - amount * (1 - c[1]), c[2]))

def estimateJitter(dataArray):
    """ creates random jitter scaled by local density of points"""
    from scipy.stats import gaussian_kde
    kde = gaussian_kde(dataArray)
    density = kde(dataArray)
    jitter = np.random.randn(len(dataArray))*density
    return jitter
pBGMenu

In [ ]:
#reapply plotting style
applyPlotStyle(pBGMenu.value)

In [ ]:
# Created csv manually during analysis
dPath = "/Users/angueyraaristjm/Documents/eelImaging/tempAnalysis/CRnr2e3F0s/xOGaCT/"
fName = "CRnr2e3F0s_xOGaCT_Counts.csv"

dPath = "/Users/angueyraaristjm/Library/CloudStorage/OneDrive-NationalInstitutesofHealth/zf/Analysis/CRnr2e3F0s/m2Gs2C/"
dPath = "/Users/angueyraaristjm/Documents/eelImaging/tempAnalysis/CRnr2e3F0s/m2Gs2C/"
fName = "CRnr2e3F0s_m2Gs2C_Counts.csv"

df = pd.read_csv(dPath + fName)
df

In [ ]:
geneList = ['wt','nr2e3']
nGenes = np.size(geneList)
# photoreceptors = ['R','U','S','M','L']
photoreceptors = ['S','M']

plotname = ''
fH, axH = plt.subplots(figsize= [12,8])

barW = nGenes+2; # bar width
barD = nGenes+1; # bar distance whitin 1 photoreceptor subtype
barP = -nGenes+1; # position in x-axis
barStep = 0.9; # distance between photoreceptor groups
barPos = []; #array to save bar positions


j=barP;
for gene in geneList:
    i=0;
    j=j+1;
    for pr in photoreceptors:
        i = i+barStep;
        countData = df[(df['genotype']==gene) & (df['excludeFlag']==False)][pr]
        jitter = np.random.randn(len(countData))*0.01
#         textindent = 1/(2.5*barD);
        textindent = 0.10;
        pH = plt.bar([i+j/barD], np.mean(countData), width=1/barW, color=lighten_color(zfC[pr],1), linewidth = 2, edgecolor = lighten_color(zfC[pr],1)); #color=zfC[pr], 
#         pH = plt.bar([i+j/barD], np.mean(countData), yerr = sem(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr][~np.isnan(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr])], ddof = 0), align='center', ecolor='black', capsize=10, width=1/barW, color=zfC[pr], edgecolor = zfC[pr]);
#         pH = plt.bar([i+j/barD], np.mean(countData), yerr = np.std(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr][~np.isnan(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr])], ddof = 0), align='center', ecolor='black', capsize=10, width=1/barW, color=zfC[pr], edgecolor = zfC[pr]);
       # pH = plt.bar([i+j/barD], np.mean(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), yerr = np.std(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), ecolor=lighten_color(zfC[pr],1.2), capsize=10, width=1/barW, color=zfC[pr], edgecolor = zfC[pr]);
        #pH = plt.errorbar([i+j/barD], np.mean(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), yerr = np.std(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), ecolor=lighten_color(zfC[pr],1.1), elinewidth=2, capsize=5, capthick=2, barsabove= True)
#         pH = plt.scatter(np.ones(len(countData))*[i+j/barD]+jitter, countData, color=lighten_color(zfC[pr],2/3), zorder=2, marker = zfGm[gene], edgecolor='#ffffff', linewidth=0.5, alpha = 0.4);
        pH = plt.errorbar(i+j/barD, np.mean(countData), yerr = [[0],[np.std(countData)]], ecolor=lighten_color(zfC[pr],1), elinewidth=3, capsize=8, capthick=3, zorder=9)
        # plt.text((i+j/barD)-textindent, 3, prLabel[pr], font_properties=font_prop, fontsize=20, ha='left', alpha=0.6)
        pH = plt.scatter(np.ones(len(countData))*[i+j/barD]+jitter, countData, color='#ffffff', zorder=8, marker = 'o', s=50, edgecolor='#ffffff', linewidth=0.5, alpha = .6);
        barPos = np.append(barPos,(i+j/barD))

formatFigure(fH, axH, pH)
axH.set_xticks(np.sort(barPos));

axH.set_xticklabels(geneList * len(photoreceptors));
# axH.set_ylim([0,500]); # this was 400

# savePath = 'C:/Users/pataklk/OneDrive - National Institutes of Health/zf/F0_Analysis/CRfoxq2F0/'
# savePath = "/Users/angueyraaristjm/OneDrive - National Institutes of Health/zf/F0_Analysis/CRfoxq2F0/"
# savePath = "/Users/angueyraaristjm/Documents/LiLab/Presentations/revealjs/resources/20211008_UCLA/"
# plt.savefig(savePath + "Counts_foxq2.png", transparent=True, format="png")
# plt.savefig(savePath + "Counts_foxq2_B.svg", transparent=True, format="svg")